In [1]:
import pandas as pd
import numpy as np

import altair as alt
%load_ext autotime

In [2]:
# inputs
num_records = 10000


# initalize df
order_df = pd.DataFrame({"CUSTOMER_ID": np.random.randint(1, 100, size=(num_records)),
                         "ORDER_ID": np.random.randint(1, 7, size=(num_records)), 
                         "PROD_ID": np.random.randint(1, 1000, size=(num_records)), 
                         "ORDER_QTY": np.random.randint(1, 20, size=(num_records))
                        })
print(f"Number of Rows: {order_df.shape[0]}")
order_df.head()

Number of Rows: 10000


,CUSTOMER_ID,ORDER_ID,PROD_ID,ORDER_QTY
0,46,1,161,6
1,98,5,564,8
2,2,5,134,6
3,80,4,37,8
4,89,1,968,3


time: 14.1 ms


In [3]:
# Check if order df needs to be padded
padCheck = order_df.groupby(['CUSTOMER_ID','ORDER_ID'])['PROD_ID'].count()
padCheck

CUSTOMER_ID  ORDER_ID
1            1           18
             2           17
             3           15
             4           13
             5           17
             6           13
2            1           15
             2           25
             3           22
             4           14
             5           13
             6           21
3            1           12
             2           18
             3           13
             4           19
             5           23
             6           11
4            1           12
             2           13
             3           27
             4           22
             5           14
             6           16
5            1           16
             2           15
             3           14
             4           20
             5           13
             6           24
                         ..
95           1            9
             2           17
             3            7
             4           1

time: 9.22 ms


In [5]:
# Pad the order df
padded_order_df = pad_order_df(order_df)



Customer ID: 1
Number of Unique Prod Id's for a customer 92

Order Id: 1 has 18 Prod Id's
Number of Missing Prod Id's : 74
Padding with: 74
Padded Dataframe size (74, 4)

Order Id: 2 has 17 Prod Id's
Number of Missing Prod Id's : 75
Padding with: 75
Padded Dataframe size (75, 4)

Order Id: 3 has 15 Prod Id's
Number of Missing Prod Id's : 77
Padding with: 77
Padded Dataframe size (77, 4)

Order Id: 4 has 13 Prod Id's
Number of Missing Prod Id's : 79
Padding with: 79
Padded Dataframe size (79, 4)

Order Id: 5 has 17 Prod Id's
Number of Missing Prod Id's : 75
Padding with: 75
Padded Dataframe size (75, 4)

Order Id: 6 has 13 Prod Id's
Number of Missing Prod Id's : 79
Padding with: 79
Padded Dataframe size (79, 4)

Customer ID: 2
Number of Unique Prod Id's for a customer 104

Order Id: 1 has 14 Prod Id's
Number of Missing Prod Id's : 90
Padding with: 90
Padded Dataframe size (90, 4)

Order Id: 2 has 25 Prod Id's
Number of Missing Prod Id's : 79
Padding with: 79
Padded Dataframe size (79, 

In [6]:
# Check if order df needs to be padded
padCheck = padded_order_df.groupby(['CUSTOMER_ID','ORDER_ID'])['PROD_ID'].count()
padCheck

CUSTOMER_ID  ORDER_ID
1            1            92
             2            92
             3            92
             4            92
             5            92
             6            92
2            1           105
             2           104
             3           104
             4           104
             5           104
             6           104
3            1            89
             2            89
             3            89
             4            89
             5            90
             6            89
4            1            99
             2            99
             3            99
             4            99
             5            99
             6            99
5            1            97
             2            96
             3            96
             4            96
             5            96
             6            96
                        ... 
95           1            87
             2            88
             3       

time: 11.8 ms


In [4]:
def pad_order_df(order_df):
    """
    Pads order id by unique prod id. 
    Input: Order detail dataframe
    Output: padded Order detail dataframe.
    Notes:
        - Output shape needs to be larger than original
    """


    # 1.  Split order detail by customer id
    grpdCustId_df = order_df.groupby(['CUSTOMER_ID'])

    # 2.  Loop through each customer ids and pad order ids with missing prod id's
    for k, custId_df in grpdCustId_df:
        print(f"\nCustomer ID: {custId_df['CUSTOMER_ID'].unique()[0]}")

        # 3. Get unique prod ids for a customer 
        unique_prodCust_Ids = custId_df['PROD_ID'].unique()

        print(f"Number of Unique Prod Id's for a customer {len(unique_prodCust_Ids)}")


        # 4.  Group by Order Id 
        grpd_orderIds = custId_df.groupby(['ORDER_ID'])

        for k1, orderId_df in grpd_orderIds:
            order_id = orderId_df['ORDER_ID'].unique()[0]
            #print(f" Processing Order Id:  {k1}")

            # 5. Get the unique Prod Id's for the order Id
            unique_prodOrderId = orderId_df['PROD_ID'].unique()
            print(f"\nOrder Id: {k1} has {len(unique_prodOrderId)} Prod Id's")

            # 6. Subtract the unique ids from the order id 
            missing_prod_Ids = (list(set(unique_prodCust_Ids) - set(unique_prodOrderId)))

            num_missing_prod_Ids = len(missing_prod_Ids)
            print(f"Number of Missing Prod Id's : {num_missing_prod_Ids}")

            # 6. Check if there are any Prod Ids to padd for an Order Id
            if num_missing_prod_Ids > 0:
                print(f"Padding with: {num_missing_prod_Ids}")

                # 7. Make a new dataframe with the just unique ids
                df_fill  = pd.DataFrame(np.nan,
                                        index=np.arange(num_missing_prod_Ids), 
                                        columns=order_df.columns)
                # 8. Fill na's
                df_fill['CUSTOMER_ID'] = orderId_df['CUSTOMER_ID'].unique()[0]
                df_fill['ORDER_ID'] = order_id
                df_fill['PROD_ID'] = missing_prod_Ids
                print(f"Padded Dataframe size {df_fill.shape}")

                # 9. Add the fill df to the order df
                order_df = order_df.append(df_fill, ignore_index=True, sort=False)
    
    return order_df


time: 3.1 ms


In [ ]:
padCheck = padded_order_df.groupby(['CUSTOMER_ID','ORDER_ID'])['PROD_ID'].count()
padCheck